In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os 
from pathlib import Path
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#setting Path
image_dir = Path('/content/gdrive/My Drive/Fish4Knowledge')
#get filepaths of all files
filepaths = list(image_dir.glob(r'**/*.png'))

In [ ]:
data = []
#To get filepaths and label
for i in range(len(filepaths)):
    label = os.path.split(os.path.split(filepaths[i])[0])[1]
    data.append([str(filepaths[i]), label])

#creating DataFrame
df = pd.DataFrame(data, columns=['path', 'label'])
#droping Non Ground truths
df = df[df['label'].apply(lambda x: x[0:4] != "mask")].reset_index(drop = True)

In [ ]:
#classes
df.label.unique()

array(['fish_23', 'fish_02', 'fish_22', 'fish_21', 'fish_20', 'fish_19',
       'fish_18', 'fish_17', 'fish_16', 'fish_15', 'fish_14', 'fish_13',
       'fish_12', 'fish_11', 'fish_10', 'fish_09', 'fish_08', 'fish_07',
       'fish_06', 'fish_05', 'fish_01', 'fish_03', 'fish_04'],
      dtype=object)

In [ ]:
#checking Null values
df.isnull().any()

path     False
label    False
dtype: bool

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
#Handling Image data Generators
train_data_gen = ImageDataGenerator(validation_split=0.25)
test_data_gen = ImageDataGenerator()

train_imgs = train_data_gen.flow_from_dataframe(dataframe=train_data, x_col='path', y_col='label', target_size=(200,200),class_mode="categorical", color_mode='grayscale',batch_size=24, shuffle=True,subset="training")
val_imgs = train_data_gen.flow_from_dataframe(dataframe=train_data, x_col='path', y_col='label', target_size=(200,200), class_mode="categorical", color_mode='grayscale',batch_size=24, shuffle=True,subset="validation")

test_imgs = test_data_gen.flow_from_dataframe(dataframe=test_data,x_col='path', y_col='label', target_size=(200,200), class_mode="categorical", color_mode='grayscale',batch_size=24, shuffle=False)

Found 18494 validated image filenames belonging to 23 classes.
Found 6164 validated image filenames belonging to 23 classes.
Found 2740 validated image filenames belonging to 22 classes.


In [ ]:
#Dividing Data into Train, Test, and Validation
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size = 0.1, random_state = 42)
train_data.count()
test_data.count()

path     2740
label    2740
dtype: int64

In [ ]:
from keras import models
from keras.models import Sequential
from keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten,MaxPooling2D, InputLayer
from keras.layers import Dense, GlobalAveragePooling2D, Activation, Reshape, Permute, multiply

In [ ]:
#CNN
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', input_shape=(200,200,1)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3,3), padding='same'))


model.add(BatchNormalization())
#SE Block
#inio Squeeze 
model1=Sequential()
model1.add(GlobalAveragePooling2D()(model))
model1.add(Reshape((1,1,128))(model1))
model1.add(Flatten())
#FC layer1
model1.add(Dense(units=8,activation="relu")(model1))

#Excitation
model1.add(Dense(units=128,activation="sigmoid")(model1)) #FC layer2
model.add(multiply([model,model1]))


model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))



model.add(Flatten())
model.add(Dense(256))
model.add(BatchNormalization()(model))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(23))
model.add(BatchNormalization()(model))
model.add(Activation('softmax'))

ValueError: ignored

In [ ]:
from keras.layers import Dense, GlobalAveragePooling2D, Activation, Reshape, Permute, multiply


def SEBlock(se_ratio = 16, activation = "relu", data_format = 'channels_last', ki = "he_normal"):
    '''
    se_ratio : ratio for reduce the filter number of first Dense layer(fc layer) in block
    activation : activation function that of first dense layer
    data_format : channel axis is at the first of dimension or the last
    ki : kernel initializer
    '''

    def f(input_x):

        channel_axis = -1 if data_format == 'channels_last' else 1
        input_channels = input_x._keras_shape[channel_axis]

        reduced_channels = input_channels // se_ratio

        #Squeeze operation
        x = GlobalAveragePooling2D()(input_x)
        x = Reshape(1,1,input_channels)(x) if data_format == 'channels_first' else x
        x = Dense(reduced_channels, kernel_initializer= ki)(x)
        x = Activation(activation)(x)
        #Excitation operation
        x = Dense(input_channels, kernel_initializer=ki, activation='sigmoid')(x)
        x = Permute(dims=(3,1,2))(x) if data_format == 'channels_first' else x
        x = multiply([input_x, x])

        return x

    return f

In [ ]:
from tensorflow.keras.optimizers import *
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics = ['accuracy', 'AUC'])

TypeError: ignored